In [19]:
#!pip install category_encoders

In [1]:
#!pip install xgboost

In [27]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.metrics import f1_score, accuracy_score
import category_encoders as ce
import xgboost as xgb

In [3]:
df=pd.read_csv('Logfile1.csv', usecols=[1,2,3,4,5,6])
df.shape

(42958, 6)

In [4]:
df.dropna(inplace=True)
df.shape

(35330, 6)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 35330 entries, 0 to 42952
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Process Name  35330 non-null  object
 1   PID           35330 non-null  int64 
 2   Operation     35330 non-null  object
 3   Path          35330 non-null  object
 4   Result        35330 non-null  object
 5   Detail        35330 non-null  object
dtypes: int64(1), object(5)
memory usage: 1.9+ MB


In [6]:
df.head()

,Process Name,PID,Operation,Path,Result,Detail
0,svchost.exe,4864,RegQueryKey,HKCU\Software\Classes,SUCCESS,Query: Name
1,svchost.exe,4864,RegQueryKey,HKCU\Software\Classes,SUCCESS,"Query: HandleTags, HandleTags: 0x0"
2,svchost.exe,3516,LockFile,C:\ProgramData\Microsoft\Windows\AppRepository...,SUCCESS,"Exclusive: False, Offset: 124, Length: 1, Fail..."
3,svchost.exe,4864,RegQueryKey,HKCU\Software\Classes,SUCCESS,"Query: HandleTags, HandleTags: 0x0"
4,svchost.exe,3516,UnlockFileSingle,C:\ProgramData\Microsoft\Windows\AppRepository...,SUCCESS,"Offset: 124, Length: 1"


In [10]:
X=df[['PID','Operation','Path','Result','Detail']]
Y=df['Process Name']
print(X.shape)
print(Y.shape)

(35330, 5)
(35330,)


In [12]:
Y.nunique()

19

In [22]:
le = preprocessing.LabelEncoder()
label_encoder = le.fit(Y)
Y = label_encoder.transform(Y)
Y[35:40]

array([2, 2, 2, 2, 2])

In [23]:
Y[30:40]

array([16, 16, 16, 16,  2,  2,  2,  2,  2,  2])

In [24]:
numeric_features = X.select_dtypes(include=['int64', 'float64']).columns
categorical_features = X.select_dtypes(include=['object']).columns
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, stratify=Y)

In [ ]:
xgb_model = xgb.XGBClassifier(objective="multi:softmax", random_state=42, eval_metric="mlogloss", num_class=Y.nunique(),random_state=42)

In [25]:
encoder_list = [ce.backward_difference.BackwardDifferenceEncoder, 
               ce.basen.BaseNEncoder,
               ce.binary.BinaryEncoder,
                ce.cat_boost.CatBoostEncoder,
                ce.hashing.HashingEncoder,
                ce.helmert.HelmertEncoder,
                ce.james_stein.JamesSteinEncoder,
                ce.one_hot.OneHotEncoder,
                ce.leave_one_out.LeaveOneOutEncoder,
                ce.m_estimate.MEstimateEncoder,
                ce.ordinal.OrdinalEncoder,
                ce.polynomial.PolynomialEncoder,
                ce.sum_coding.SumEncoder,
                ce.target_encoder.TargetEncoder,
                ce.woe.WOEEncoder
                ]


for encoder in encoder_list:
    
    numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())])
    categorical_transformer = Pipeline(steps=[
    ('woe', encoder())])
    
    preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])
    
    pipe = Pipeline(steps=[('preprocessor', preprocessor),
                      ('classifier', RandomForestClassifier(n_estimators=50))])
    
    model = pipe.fit(X_train, y_train)
    
    y_pred = model.predict(X_test)
    print(encoder)
    print(f1_score(y_test, y_pred, average='macro'))

<class 'category_encoders.backward_difference.BackwardDifferenceEncoder'>
0.8730402004112453
<class 'category_encoders.basen.BaseNEncoder'>
0.8853470839793918
<class 'category_encoders.binary.BinaryEncoder'>
0.8875542967530675
<class 'category_encoders.cat_boost.CatBoostEncoder'>
0.9862350037487864
<class 'category_encoders.hashing.HashingEncoder'>
0.9770129626825843
<class 'category_encoders.helmert.HelmertEncoder'>
0.8687323723281826
<class 'category_encoders.james_stein.JamesSteinEncoder'>
0.9733696069553055
<class 'category_encoders.one_hot.OneHotEncoder'>
0.9096541245899241
<class 'category_encoders.leave_one_out.LeaveOneOutEncoder'>
0.6735528007046138
<class 'category_encoders.m_estimate.MEstimateEncoder'>
0.9826049263307013
<class 'category_encoders.ordinal.OrdinalEncoder'>
0.9142510819528228


C:\Users\pcvadmin\anaconda3\envs\dlib_py37\lib\site-packages\patsy\contrasts.py:266: RuntimeWarning: overflow encountered in power
  raw_poly = scores.reshape((-1, 1)) ** np.arange(n).reshape((1, -1))
C:\Users\pcvadmin\anaconda3\envs\dlib_py37\lib\site-packages\patsy\contrasts.py:268: RuntimeWarning: invalid value encountered in sign
  q *= np.sign(np.diag(r))


ValueError: Input contains NaN, infinity or a value too large for dtype('float32').

In [34]:
xgb_model = xgb.XGBClassifier(objective="multi:softmax",n_estimators=50, random_state=42,learning_rate=0.3, eval_metric="mlogloss", num_class=df['Process Name'].nunique())
encoder_list = [ce.backward_difference.BackwardDifferenceEncoder, 
               ce.basen.BaseNEncoder,
               ce.binary.BinaryEncoder,
                ce.cat_boost.CatBoostEncoder,
                ce.hashing.HashingEncoder,
                ce.helmert.HelmertEncoder,
                ce.james_stein.JamesSteinEncoder,
                ce.one_hot.OneHotEncoder,
                ce.leave_one_out.LeaveOneOutEncoder,
                ce.m_estimate.MEstimateEncoder,
                ce.ordinal.OrdinalEncoder,
                ce.sum_coding.SumEncoder,
                ce.target_encoder.TargetEncoder,
                ce.woe.WOEEncoder  # for this target values should be binary
                ]


for encoder in encoder_list:
    
    numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())])
    categorical_transformer = Pipeline(steps=[
    ('woe', encoder())])
    
    preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])
    
    pipe = Pipeline(steps=[('preprocessor', preprocessor),
                      ('classifier', xgb_model)])
    
    model = pipe.fit(X_train, y_train)
    
    y_pred = model.predict(X_test)
    print(encoder)
    print(accuracy_score(y_test, y_pred))

<class 'category_encoders.backward_difference.BackwardDifferenceEncoder'>
0.9998584772148316
<class 'category_encoders.basen.BaseNEncoder'>
0.9998584772148316
<class 'category_encoders.binary.BinaryEncoder'>
0.9998584772148316
<class 'category_encoders.cat_boost.CatBoostEncoder'>
0.9998584772148316
<class 'category_encoders.hashing.HashingEncoder'>
0.9998584772148316
<class 'category_encoders.helmert.HelmertEncoder'>
0.9998584772148316
<class 'category_encoders.james_stein.JamesSteinEncoder'>
0.9998584772148316
<class 'category_encoders.one_hot.OneHotEncoder'>
0.9998584772148316
<class 'category_encoders.leave_one_out.LeaveOneOutEncoder'>
0.9974525898669686
<class 'category_encoders.m_estimate.MEstimateEncoder'>
0.9998584772148316
<class 'category_encoders.ordinal.OrdinalEncoder'>
0.9998584772148316
<class 'category_encoders.sum_coding.SumEncoder'>
0.9998584772148316
<class 'category_encoders.target_encoder.TargetEncoder'>
0.9998584772148316


ValueError: The target column y must be binary. But the target contains 19 unique value(s).

In [31]:
df['Process Name'].nunique()

19

In [33]:
xgb.XGBClassifier?

In [37]:
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import VotingClassifier


kfold = model_selection.KFold(n_splits=5, random_state=42, shuffle=True)
# create the sub models
estimators = []
model1 =  RandomForestClassifier(n_estimators=50)
estimators.append(('RandomForest', model1))
model2 = xgb.XGBClassifier(objective="multi:softmax",n_estimators=50,
                           learning_rate=0.3, eval_metric="mlogloss", num_class=df['Process Name'].nunique())
estimators.append(('xgboost', model2))

# create the ensemble model
ensemble = VotingClassifier(estimators)

for encoder in encoder_list:

    numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())])
    categorical_transformer = Pipeline(steps=[
    ('woe', encoder())])
    
    preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])
    pipe = Pipeline(steps=[('preprocessor', preprocessor),
                          ('classifier', ensemble)])

    model = pipe.fit(X_train, y_train)
    y_pred = model.predict(X_test) 
    results = model_selection.cross_val_score(ensemble, X_train, y_train, cv=kfold)  # preprocessor.fit(X_train)
    print(encoder)
    print(" cross-validation score is {:.3f} ".format(results.mean()))
    print(" Accuracy - score : {:.5f}".format(accuracy_score(y_test, y_pred)))

C:\Users\pcvadmin\anaconda3\envs\dlib_py37\lib\site-packages\sklearn\model_selection\_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: could not convert string to float: 'RegOpenKey'

  FitFailedWarning)
C:\Users\pcvadmin\anaconda3\envs\dlib_py37\lib\site-packages\sklearn\model_selection\_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: could not convert string to float: 'RegQueryKey'

  FitFailedWarning)
C:\Users\pcvadmin\anaconda3\envs\dlib_py37\lib\site-packages\sklearn\model_selection\_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: could not convert string to float: 'RegOpenKey'

  FitFailedWarning)
C:\Users\pcvadmin\anaconda3\envs\dlib_py37\lib\site

<class 'category_encoders.backward_difference.BackwardDifferenceEncoder'>
 cross-validation score is nan 
 Accuracy - score : 0.98783


C:\Users\pcvadmin\anaconda3\envs\dlib_py37\lib\site-packages\sklearn\model_selection\_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: could not convert string to float: 'RegOpenKey'

  FitFailedWarning)
C:\Users\pcvadmin\anaconda3\envs\dlib_py37\lib\site-packages\sklearn\model_selection\_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: could not convert string to float: 'RegQueryKey'

  FitFailedWarning)
C:\Users\pcvadmin\anaconda3\envs\dlib_py37\lib\site-packages\sklearn\model_selection\_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: could not convert string to float: 'RegOpenKey'

  FitFailedWarning)
C:\Users\pcvadmin\anaconda3\envs\dlib_py37\lib\site

<class 'category_encoders.basen.BaseNEncoder'>
 cross-validation score is nan 
 Accuracy - score : 0.99575


C:\Users\pcvadmin\anaconda3\envs\dlib_py37\lib\site-packages\sklearn\model_selection\_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: could not convert string to float: 'RegOpenKey'

  FitFailedWarning)
C:\Users\pcvadmin\anaconda3\envs\dlib_py37\lib\site-packages\sklearn\model_selection\_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: could not convert string to float: 'RegQueryKey'

  FitFailedWarning)
C:\Users\pcvadmin\anaconda3\envs\dlib_py37\lib\site-packages\sklearn\model_selection\_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: could not convert string to float: 'RegOpenKey'

  FitFailedWarning)
C:\Users\pcvadmin\anaconda3\envs\dlib_py37\lib\site

...

C:\Users\pcvadmin\anaconda3\envs\dlib_py37\lib\site-packages\sklearn\model_selection\_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: could not convert string to float: 'RegOpenKey'

  FitFailedWarning)
C:\Users\pcvadmin\anaconda3\envs\dlib_py37\lib\site-packages\sklearn\model_selection\_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: could not convert string to float: 'RegQueryKey'

  FitFailedWarning)
C:\Users\pcvadmin\anaconda3\envs\dlib_py37\lib\site-packages\sklearn\model_selection\_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: could not convert string to float: 'RegOpenKey'

  FitFailedWarning)
C:\Users\pcvadmin\anaconda3\envs\dlib_py37\lib\site

<class 'category_encoders.cat_boost.CatBoostEncoder'>
 cross-validation score is nan 
 Accuracy - score : 0.99929


C:\Users\pcvadmin\anaconda3\envs\dlib_py37\lib\site-packages\sklearn\model_selection\_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: could not convert string to float: 'RegOpenKey'

  FitFailedWarning)
C:\Users\pcvadmin\anaconda3\envs\dlib_py37\lib\site-packages\sklearn\model_selection\_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: could not convert string to float: 'RegQueryKey'

  FitFailedWarning)
C:\Users\pcvadmin\anaconda3\envs\dlib_py37\lib\site-packages\sklearn\model_selection\_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: could not convert string to float: 'RegOpenKey'

  FitFailedWarning)
C:\Users\pcvadmin\anaconda3\envs\dlib_py37\lib\site

<class 'category_encoders.hashing.HashingEncoder'>
 cross-validation score is nan 
 Accuracy - score : 0.99844


KeyboardInterrupt: 

In [39]:
temp=preprocessor.fit_transform(X_train)
temp

array([[-0.47944806,  1.        , -1.        , ..., -1.        ,
        -1.        , -1.        ],
       [-0.47944806,  1.        ,  1.        , ..., -1.        ,
        -1.        , -1.        ],
       [ 0.16441019,  1.        ,  1.        , ..., -1.        ,
        -1.        , -1.        ],
       ...,
       [ 0.16441019,  1.        , -1.        , ..., -1.        ,
        -1.        , -1.        ],
       [-0.47944806,  1.        , -1.        , ..., -1.        ,
        -1.        , -1.        ],
       [ 0.05189127,  1.        ,  0.        , ..., -1.        ,
        -1.        , -1.        ]])